In [1]:
import pandas as pd
import sqlite3

## SQL Hepler Functions

In [ ]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Create Table
    sql_string = """CREATE TABLE {} ({});""".format(table_name, cols_string)
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = """INSERT INTO {} ({}) VALUES ({});""".format(table_name, 
                                                               cols_string,
                                                               val_wildcard_string)
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} uploaded to {table_name}')

    con.commit()
    con.close()


def sql_query_to_pd(sql_text: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_text (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Execute the SQL query
    cursor = con.execute(sql_text)

    # Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Close the connection
    con.close()

    # Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

## Execute Query

In [ ]:
# Read the csv file into a dataframe
# Dataset from https://www.kaggle.com/gpreda/covid-world-vaccination-progress
input_df = pd.read_csv('country_vaccinations.csv')

# Upload the dataframe to a SQL Table
pd_to_sqlDB(input_df,
            table_name='country_vaccinations',
            db_name='default.db')

# Exectue a SQL query
sql_string = """
    SELECT country, SUM(total_vaccinations) as total_vaccinated
    FROM country_vaccinations 
    WHERE total_vaccinations IS NOT NULL 
    GROUP BY country
    ORDER BY total_vaccinated DESC
"""

result_df = sql_query_to_pd(sql_string, db_name='default.db')
print(result_df)